The data frame formatting is missing when you view on GitHub.\
View it using [nbviewer](https://nbviewer.org/github/phiSgr/rektdeal/blob/main/examples/gazzilli_weak_response.ipynb) instead.

North opens 1S with 18-19 balanced.

```
1S 1N
2C 2S
```

The 2C bid is Gazzilli, which shows a strong hand or a normal hand with clubs.
This allows responder to respond with weaker hands without pushing the bidding too high.
With the 2S bid, South is either very weak (3-5) with 3-card support,
or weak (5-7) without support.

Should North invite?

In [1]:
// Because of class loading reasons,
// this has to be loaded before or at the same cell as ReKtDeal
// for the `toFormattedFrame` and `toDataFrame` methods to work.
%use dataframe

@file:DependsOn("com.github.phisgr:rektdeal:0.1.0")

import com.github.phisgr.dds.*
import com.github.phisgr.dds.Deal as DdsDeal
import com.github.phisgr.rektdeal.*

println("We will launch $threadCount threads to generate and solve the deals.")

We will launch 14 threads to generate and solve the deals.


In [2]:
val bal1S = Shape("5(332)")
val flat = Shape("(4333)")

val smartStack = SmartStack(bal1S, Evaluator.hcp, 18..19)

val contract2S = Contract("2S")
val contract3S = Contract("3S")
val contract4S = Contract("4S")

val contract2N = Contract("2N")
val contract3N = Contract("3N")

In [3]:
data class DealTricks(val deal: Deal, val spadeTricks: Int, val ntTricks: Int)

In the [introduction](./introduction.ipynb),
you can see an example of `multiThread` that updates some shared state (namely the `AtomicIntegerArray`).

Programming with mutable shared state is often not easy.

Here is a slightly different version of the `multiThread` function,
which has a state object for each thread.

In [4]:
log("started.")
val multiThreadResults: List<List<DealTricks>> = multiThread(
    count = 100_000,
    dealer = { Dealer(N = smartStack) },
    state = { mutableListOf<DealTricks>() },
    accept = { deal ->
        deal.south.spades.size == 3 && deal.south.hcp in 3..5 || // south very weak with support
            deal.south.spades.size < 3 && deal.south.hcp in 5..7 // south weak no support
    },
    action = { dealCount, deal, state ->
        if (dealCount % 5000 == 0) {
            log("$dealCount deals analyzed.")
        }

        val spadeTricks = deal.ddTricks(S, NORTH)
        val ntTricks = deal.ddTricks(N, SOUTH)

        state += DealTricks(deal, spadeTricks = spadeTricks, ntTricks = ntTricks)
    }
)

23:38:34 started.
23:38:43 5000 deals analyzed.
23:38:51 10000 deals analyzed.
23:39:00 15000 deals analyzed.
23:39:09 20000 deals analyzed.
23:39:18 25000 deals analyzed.
23:39:27 30000 deals analyzed.
23:39:37 35000 deals analyzed.
23:39:47 40000 deals analyzed.
23:39:56 45000 deals analyzed.
23:40:06 50000 deals analyzed.
23:40:16 55000 deals analyzed.
23:40:26 60000 deals analyzed.
23:40:35 65000 deals analyzed.
23:40:45 70000 deals analyzed.
23:40:55 75000 deals analyzed.
23:41:04 80000 deals analyzed.
23:41:13 85000 deals analyzed.
23:41:23 90000 deals analyzed.
23:41:32 95000 deals analyzed.
23:41:42 100000 deals analyzed.


`multiThreadResults` is a list of list of `DealTricks` — a list for each thread, each containing about `count / threadSize` deals.

To make it easier to work with we flatten it into a [lazy `Sequence`](https://kotlinlang.org/docs/sequences.html).

In [5]:
println(multiThreadResults.map { it.size })
val results = multiThreadResults.asSequence().flatten()

[7194, 7062, 7397, 7180, 6991, 7169, 7043, 7143, 7186, 7133, 7160, 7144, 7062, 7136]


In [6]:
val strategies = listOf(
    "pass",     // always pass
    "inv",      // always invite
    "inv 4S",   // always invite, south bids 4S when max (i.e. 5HCP)
    "inv w/19", // invite with 19HCP, pass with 18
    "19 4S",    // invite with 19HCP, south bids 4S when max (i.e. 5HCP)
)
val vul = PayOff(strategies, PayOff.impFromScores)
val nonVul = PayOff(strategies, PayOff.impFromScores)

results.forEach { (deal, spadeTrics, ntTricks) ->
    listOf(true to vul, false to nonVul).forEach { (isVul, payOff) ->

        val score2S = contract2S.score(spadeTrics, vulnerable = isVul)
        val score3S = contract3S.score(spadeTrics, vulnerable = isVul)
        val score4S = contract4S.score(spadeTrics, vulnerable = isVul)

        val score2N = contract2N.score(ntTricks, vulnerable = isVul)
        val score3N = contract3N.score(ntTricks, vulnerable = isVul)

        val pass = score2S
        val inv = when {
            // if south is very weak with support, retreat to 3S
            deal.south.spades.size == 3 -> score3S
            // with minimum, we pass
            deal.south.hcp == 5 -> score2N
            else -> score3N
        }


        val northMax = deal.north.hcp == 19
        val southSupportMax = deal.south.spades.size == 3 && deal.south.hcp == 5 && !flat(deal.south)

        val inv4S = if (southSupportMax) score4S else inv

        val invWith19HCP = if (northMax) inv else pass
        val invWith19Maybe4S = if (northMax && southSupportMax) score4S else invWith19HCP
        payOff.addData(
            intArrayOf(
                pass,
                inv,
                inv4S,
                invWith19HCP,
                invWith19Maybe4S,
            )
        )
    }
}


In [7]:
vul

        pass    inv     inv 4S  inv w/1 19 4S   
pass            -0.56   -0.65   -0.55   -0.64   
                (0.02)  (0.02)  (0.01)  (0.01)  
inv     +0.56           -0.07   +0.01   -0.07   
        (0.02)          (0.01)  (0.02)  (0.02)  
inv 4S  +0.65   +0.07           +0.09   +0.01   
        (0.02)  (0.01)          (0.02)  (0.02)  
inv w/1 +0.55   -0.01   -0.09           -0.08   
        (0.01)  (0.02)  (0.02)          (0.00)  
19 4S   +0.64   +0.07   -0.01   +0.08           
        (0.01)  (0.02)  (0.02)  (0.00)          


The `PayOff` reporting is designed for terminal output,
and unfortunately the strategy name `inv w/19` was chopped off.
Here in Jupyter notebook we can do better.

In [8]:
vul.toFormattedFrame()

strategy,pass,inv,inv 4S,inv w/19,19 4S
pass,null,-0.56 ± 0.02,-0.65 ± 0.02,-0.55 ± 0.01,-0.64 ± 0.01
inv,+0.56 ± 0.02,null,-0.07 ± 0.01,+0.01 ± 0.02,-0.07 ± 0.02
inv 4S,+0.65 ± 0.02,+0.07 ± 0.01,null,+0.09 ± 0.02,+0.01 ± 0.02
inv w/19,+0.55 ± 0.01,-0.01 ± 0.02,-0.09 ± 0.02,null,-0.08 ± 0.00
19 4S,+0.64 ± 0.01,+0.07 ± 0.02,-0.01 ± 0.02,+0.08 ± 0.00,null


In [9]:
nonVul.toFormattedFrame()

strategy,pass,inv,inv 4S,inv w/19,19 4S
pass,null,-0.17 ± 0.01,-0.17 ± 0.02,-0.29 ± 0.01,-0.32 ± 0.01
inv,+0.17 ± 0.01,null,+0.02 ± 0.00,-0.12 ± 0.01,-0.15 ± 0.01
inv 4S,+0.17 ± 0.02,-0.02 ± 0.00,null,-0.12 ± 0.01,-0.15 ± 0.01
inv w/19,+0.29 ± 0.01,+0.12 ± 0.01,+0.12 ± 0.01,null,-0.03 ± 0.00
19 4S,+0.32 ± 0.01,+0.15 ± 0.01,+0.15 ± 0.01,+0.03 ± 0.00,null


When non-vulnerable, the best strategy for North is to pass with 18, invite with 19;
when vulnerable, it's a tie with always inviting - as the game bonus is higher.
South, even with only 5 HCP, should consider bidding 4S with support.

Can we do better? Maybe we should consider north's controls?
Since we stored the deals and tricks, re-analyzing is easy.

In [10]:
val strategies2 = listOf(
    "pass",     // always pass
    "inv",      // always invite
    "inv 4S",   // always invite, South bids 4S when max (i.e. 5HCP)
    "inv w/19", // invite with 19HCP, pass with 18
    "19 4S",    // invite with 19HCP, South bids 4S when max (i.e. 5HCP non-flat)
    *(5..8).flatMap { ctrl ->
        // invite with a certain number of controls
        // South may/may not bid 4S, respectively
        listOf("ctrl $ctrl", "ctrl $ctrl 4S")
    }.toTypedArray(),
)
val vul2 = PayOff(strategies2, PayOff.impFromScores)
val nonVul2 = PayOff(strategies2, PayOff.impFromScores)

results.forEach { (deal, spadeTrics, ntTricks) ->
    listOf(true to vul2, false to nonVul2).forEach { (isVul, payOff) ->

        val score2S = contract2S.score(spadeTrics, vulnerable = isVul)
        val score3S = contract3S.score(spadeTrics, vulnerable = isVul)
        val score4S = contract4S.score(spadeTrics, vulnerable = isVul)

        val score2N = contract2N.score(ntTricks, vulnerable = isVul)
        val score3N = contract3N.score(ntTricks, vulnerable = isVul)

        val pass = score2S
        val inv = when {
            // if south is very weak with support, retreat to 3S
            deal.south.spades.size == 3 -> score3S
            // with minimum, we pass
            deal.south.hcp == 5 -> score2N
            else -> score3N
        }

        val northMax = deal.north.hcp == 19
        val southSupportMax = deal.south.spades.size == 3 && deal.south.hcp == 5 && !flat(deal.south)

        val inv4S = if (southSupportMax) score4S else inv

        val invWith19HCP = if (northMax) inv else pass
        val invWith19Maybe4S = if (northMax && southSupportMax) score4S else invWith19HCP

        payOff.addData(
            intArrayOf(
                pass,
                inv,
                inv4S,
                invWith19HCP,
                invWith19Maybe4S,
                *(5..8).flatMap { ctrl ->
                    if (deal.north.controls >= ctrl) {
                        listOf(inv, inv4S)
                    } else {
                        listOf(pass, pass)
                    }
                }.toIntArray(),
            )
        )
    }
}


In [11]:
vul2.toFormattedFrame()

strategy,pass,inv,inv 4S,inv w/19,19 4S,ctrl 5,ctrl 5 4S,ctrl 6,ctrl 6 4S,ctrl 7,ctrl 7 4S,ctrl 8,ctrl 8 4S
pass,null,-0.56 ± 0.02,-0.65 ± 0.02,-0.55 ± 0.01,-0.64 ± 0.01,-0.58 ± 0.02,-0.67 ± 0.02,-0.61 ± 0.02,-0.72 ± 0.02,-0.52 ± 0.01,-0.62 ± 0.01,-0.19 ± 0.01,-0.23 ± 0.01
inv,+0.56 ± 0.02,null,-0.07 ± 0.01,+0.01 ± 0.02,-0.07 ± 0.02,-0.02 ± 0.00,-0.09 ± 0.01,-0.05 ± 0.01,-0.14 ± 0.01,+0.04 ± 0.01,-0.05 ± 0.02,+0.37 ± 0.02,+0.33 ± 0.02
inv 4S,+0.65 ± 0.02,+0.07 ± 0.01,null,+0.09 ± 0.02,+0.01 ± 0.02,+0.05 ± 0.01,-0.02 ± 0.00,+0.02 ± 0.01,-0.07 ± 0.01,+0.12 ± 0.02,+0.03 ± 0.02,+0.45 ± 0.02,+0.42 ± 0.02
inv w/19,+0.55 ± 0.01,-0.01 ± 0.02,-0.09 ± 0.02,null,-0.08 ± 0.00,-0.03 ± 0.02,-0.12 ± 0.02,-0.06 ± 0.01,-0.16 ± 0.02,+0.03 ± 0.01,-0.07 ± 0.01,+0.36 ± 0.01,+0.32 ± 0.01
19 4S,+0.64 ± 0.01,+0.07 ± 0.02,-0.01 ± 0.02,+0.08 ± 0.00,null,+0.06 ± 0.02,-0.03 ± 0.02,+0.02 ± 0.02,-0.08 ± 0.02,+0.12 ± 0.01,+0.02 ± 0.01,+0.45 ± 0.01,+0.41 ± 0.01
ctrl 5,+0.58 ± 0.02,+0.02 ± 0.00,-0.05 ± 0.01,+0.03 ± 0.02,-0.06 ± 0.02,null,-0.07 ± 0.01,-0.03 ± 0.01,-0.12 ± 0.01,+0.06 ± 0.01,-0.03 ± 0.02,+0.38 ± 0.02,+0.35 ± 0.02
ctrl 5 4S,+0.67 ± 0.02,+0.09 ± 0.01,+0.02 ± 0.00,+0.12 ± 0.02,+0.03 ± 0.02,+0.07 ± 0.01,null,+0.04 ± 0.01,-0.05 ± 0.01,+0.15 ± 0.02,+0.06 ± 0.01,+0.48 ± 0.02,+0.45 ± 0.02
ctrl 6,+0.61 ± 0.02,+0.05 ± 0.01,-0.02 ± 0.01,+0.06 ± 0.01,-0.02 ± 0.02,+0.03 ± 0.01,-0.04 ± 0.01,null,-0.09 ± 0.01,+0.09 ± 0.01,+0.00 ± 0.01,+0.42 ± 0.02,+0.39 ± 0.02
ctrl 6 4S,+0.72 ± 0.02,+0.14 ± 0.01,+0.07 ± 0.01,+0.16 ± 0.02,+0.08 ± 0.02,+0.12 ± 0.01,+0.05 ± 0.01,+0.09 ± 0.01,null,+0.19 ± 0.01,+0.10 ± 0.01,+0.52 ± 0.02,+0.49 ± 0.02
ctrl 7,+0.52 ± 0.01,-0.04 ± 0.01,-0.12 ± 0.02,-0.03 ± 0.01,-0.12 ± 0.01,-0.06 ± 0.01,-0.15 ± 0.02,-0.09 ± 0.01,-0.19 ± 0.01,null,-0.09 ± 0.01,+0.33 ± 0.01,+0.29 ± 0.01


In [12]:
nonVul2.toFormattedFrame()

strategy,pass,inv,inv 4S,inv w/19,19 4S,ctrl 5,ctrl 5 4S,ctrl 6,ctrl 6 4S,ctrl 7,ctrl 7 4S,ctrl 8,ctrl 8 4S
pass,null,-0.17 ± 0.01,-0.17 ± 0.02,-0.29 ± 0.01,-0.32 ± 0.01,-0.19 ± 0.01,-0.20 ± 0.02,-0.24 ± 0.01,-0.26 ± 0.01,-0.25 ± 0.01,-0.29 ± 0.01,-0.11 ± 0.01,-0.12 ± 0.01
inv,+0.17 ± 0.01,null,+0.02 ± 0.00,-0.12 ± 0.01,-0.15 ± 0.01,-0.02 ± 0.00,-0.00 ± 0.01,-0.07 ± 0.01,-0.07 ± 0.01,-0.08 ± 0.01,-0.11 ± 0.01,+0.06 ± 0.01,+0.05 ± 0.01
inv 4S,+0.17 ± 0.02,-0.02 ± 0.00,null,-0.12 ± 0.01,-0.15 ± 0.01,-0.03 ± 0.01,-0.02 ± 0.00,-0.08 ± 0.01,-0.09 ± 0.01,-0.08 ± 0.01,-0.11 ± 0.01,+0.07 ± 0.01,+0.05 ± 0.01
inv w/19,+0.29 ± 0.01,+0.12 ± 0.01,+0.12 ± 0.01,null,-0.03 ± 0.00,+0.10 ± 0.01,+0.10 ± 0.01,+0.05 ± 0.01,+0.03 ± 0.01,+0.04 ± 0.01,+0.00 ± 0.01,+0.18 ± 0.01,+0.17 ± 0.01
19 4S,+0.32 ± 0.01,+0.15 ± 0.01,+0.15 ± 0.01,+0.03 ± 0.00,null,+0.13 ± 0.01,+0.13 ± 0.01,+0.08 ± 0.01,+0.06 ± 0.01,+0.07 ± 0.01,+0.03 ± 0.01,+0.22 ± 0.01,+0.20 ± 0.01
ctrl 5,+0.19 ± 0.01,+0.02 ± 0.00,+0.03 ± 0.01,-0.10 ± 0.01,-0.13 ± 0.01,null,+0.01 ± 0.00,-0.05 ± 0.01,-0.06 ± 0.01,-0.06 ± 0.01,-0.09 ± 0.01,+0.08 ± 0.01,+0.07 ± 0.01
ctrl 5 4S,+0.20 ± 0.02,+0.00 ± 0.01,+0.02 ± 0.00,-0.10 ± 0.01,-0.13 ± 0.01,-0.01 ± 0.00,null,-0.06 ± 0.01,-0.06 ± 0.01,-0.06 ± 0.01,-0.09 ± 0.01,+0.09 ± 0.01,+0.08 ± 0.01
ctrl 6,+0.24 ± 0.01,+0.07 ± 0.01,+0.08 ± 0.01,-0.05 ± 0.01,-0.08 ± 0.01,+0.05 ± 0.01,+0.06 ± 0.01,null,-0.01 ± 0.00,-0.01 ± 0.01,-0.04 ± 0.01,+0.13 ± 0.01,+0.12 ± 0.01
ctrl 6 4S,+0.26 ± 0.01,+0.07 ± 0.01,+0.09 ± 0.01,-0.03 ± 0.01,-0.06 ± 0.01,+0.06 ± 0.01,+0.06 ± 0.01,+0.01 ± 0.00,null,+0.00 ± 0.01,-0.03 ± 0.01,+0.15 ± 0.01,+0.14 ± 0.01
ctrl 7,+0.25 ± 0.01,+0.08 ± 0.01,+0.08 ± 0.01,-0.04 ± 0.01,-0.07 ± 0.01,+0.06 ± 0.01,+0.06 ± 0.01,+0.01 ± 0.01,-0.00 ± 0.01,null,-0.03 ± 0.00,+0.14 ± 0.01,+0.13 ± 0.01


Now that's peculiar, in non-vul inviting with 19 HCP wins, but in vul inviting with 6 controls wins.

What about [HCP + controls](https://en.wikipedia.org/wiki/Zar_Points#Zar_high_card_points)?

What about matchpoints?

What about you run this notebook to find out?